In [1]:
import robosuite as suite
from robosuite.wrappers.gym_wrapper import GymWrapper
import gymnasium as gym
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
import time


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/ja/PycharmProjects/robosuite/robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


In [2]:
# create environment instance
env = GymWrapper(suite.make(
    env_name="Lift", # try with other tasks like "Stack" and "Door"
    robots="UR5e",  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=True,
    has_offscreen_renderer=True,
    use_object_obs=False,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="robot0_eye_in_hand",      # use "agentview" camera for observations
    camera_heights=84,                      # image height
    camera_widths=84,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500
))
# ), ['robot0_eye_in_hand_image'])

In [3]:
# from typing import Callable

# def make_env(env_id, rank: int, seed: int = 0) -> Callable:
#     """
#     Utility function for multiprocessed env.

#     :param env_id: (str) the environment ID
#     :param num_env: (int) the number of environment you wish to have in subprocesses
#     :param seed: (int) the inital seed for RNG
#     :param rank: (int) index of the subprocess
#     :return: (Callable)
#     """

#     def _init() -> gym.Env:
#         env = env_id
#         return env

#     set_random_seed(seed)
#     return _init

In [5]:
# vec_train_env = SubprocVecEnv([make_env(env, i) for i in range(16)])

In [12]:
from stable_baselines3 import PPO
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=100000, progress_bar=True, log_interval=5)
model.save('trained_model')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()

In [ ]:
obs = env.reset()[0]
for i in range(500):
    action, state = model.predict(obs)
    obs, reward, done,done, info = env.step(action)
    env.render()
    time.sleep(0.2)
    if done:
        break
env.close()

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to target thread (0x7f4d65a43f20)

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to target thread (0x7f4d65a43f20)

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to target thread (0x7f4d65a43f20)

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to target thread (0x7f4d65a43f20)

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to target thread (0x7f4d65a43f20)

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to target thread (0x7f4d65a43f20)

QObject::moveToThread: Current thread (0x7f4d65a43f20) is not the object's thread (0x7f4e0494f4c0).
Cannot move to tar

In [3]:
from stable_baselines3 import PPO
model = PPO.load('trained_model', env=env)
model.learn(total_timesteps=100000, progress_bar=True, log_interval=10)
model.save('trained_model')


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 500        |
|    ep_rew_mean          | 74.8       |
| time/                   |            |
|    fps                  | 89         |
|    iterations           | 10         |
|    time_elapsed         | 227        |
|    total_timesteps      | 20480      |
| train/                  |            |
|    approx_kl            | 0.49315557 |
|    clip_fraction        | 0.723      |
|    clip_range           | 0.2        |
|    entropy_loss         | -3.56      |
|    explained_variance   | 0.946      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0686    |
|    n_updates            | 1560       |
|    policy_gradient_loss | -0.0664    |
|    std                  | 0.401      |
|    value_loss           | 0.785      |
----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean  

In [6]:
import torch
torch.cuda.is_available()

True